In [1]:
library(lme4)
library(fixest)
library(tidyverse)
# load data
load("brain.rda")

Loading required package: Matrix

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
region <- arraymeta %>%
    dplyr::select(region.ancg, region.cb, region.dlpfc) %>%
    max.col %>%
    as.character
lab <- arraymeta %>%
    dplyr::select(lab.davis, lab.irvine, lab.michigan) %>%
    max.col %>%
    as.character
covariates <- arraymeta %>%
    dplyr::select(patient, age, sex, arrayversion) %>%
    dplyr::mutate(lab = lab, region = region) %>%
    dplyr::mutate(age.indicator = ifelse(age > 70, 1, 0)) 
is.gene <- !is.na(genemeta["chrom",])
expression.used <- expression[ ,is.gene]
colnames(expression.used) <- paste0("gene", 1:sum(is.gene))
dep.used <- unname(colnames(expression.used))
df <- cbind(covariates, expression.used)

In [3]:
mod <- fixest::feols(.[dep.used] ~ age.indicator |  sex + arrayversion + lab + region, df)

In [4]:
result.summary <- summary(mod, "compact", se = "hetero")
result.summary$lhs <- genemeta["sym", is.gene]

In [5]:
head(result.summary)

,lhs,age.indicator
,<chr>,<chr>
1,DDR1,0.025 (0.103)
2,RFC2,0.014 (0.098)
3,HSPA6,0.040 (0.109)
4,PAX8,-0.005 (0.098)
5,GUCA1A,0.005 (0.084)
6,UBA7,0.030 (0.092)
